# Setup

In [4]:
import plotly.io as pio
try:
    import google.colab
    print("Running as a Colab notebook")
    pio.renderers.default = "colab"
    %pip install transformer-lens fancy-einsum
    %pip install -U kaleido # kaleido only works if you restart the runtime. Required to write figures to disk (final cell)
except:
    print("Running as a Jupyter notebook")
    pio.renderers.default = "vscode"
    from IPython import get_ipython
    ipython = get_ipython()

Running as a Jupyter notebook


In [5]:
import torch
from fancy_einsum import einsum
from transformer_lens import HookedTransformer, HookedTransformerConfig, utils, ActivationCache
from torchtyping import TensorType as TT
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import einops
from typing import List, Union, Optional
from functools import partial
import pandas as pd
from pathlib import Path
import urllib.request
from bs4 import BeautifulSoup
from tqdm import tqdm
from datasets import load_dataset
import os
import json

os.environ["TOKENIZERS_PARALLELISM"] = "false" # https://stackoverflow.com/q/62691279
torch.set_grad_enabled(False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [6]:
!pip install circuitsvis
import circuitsvis as cv


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [7]:
pio.renderers.default='vscode'

def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [5]:
model = HookedTransformer.from_pretrained(
    "gpt2-large",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
    device=device,
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


## Generate Prompts For GPT-4 Example Generation

In [29]:
EXPLANATION_JSON_FILE_PATH = "data/head_explanation_1_nh_to_exp.json"

# Load results
with open(EXPLANATION_JSON_FILE_PATH) as f:
    nh_to_exp_str_key = json.load(f)

# The key of the response dict is a string '((a, b), c)' where a, b, c are integers. COnvert to tuple of (a, b, c).
nh_to_exp = {tuple(int(value.strip()) for value in k.strip('()').split(',')):v for k,v in nh_to_exp_str_key.items()}

# Load neuron word (inside neuron_finder_results)
with open("data/neuron_finder_results.json", "r") as f:
    neuron_finder_results = json.load(f)

neuron_to_token = {}

for layer in neuron_finder_results.keys():
    for neuron_ind in neuron_finder_results[layer].keys():
        neuron_to_token[(int(layer), int(neuron_ind))] = neuron_finder_results[layer][neuron_ind][0]

# We have nh_to_exp, which is a dict of the form (layer, neuron, head) -> explanation string
# and neuron_to_token, which is a dict of the form (layer, neuron) -> token

In [9]:
nh_to_exp[(31, 988, 558)]

'contains a phrase where a specific term, concept, or mechanism is introduced and then immediately defined or referred to using the term "called." In other words, the attention head is activated when there is a direct association between a previously mentioned item and the term "called." \n\nIn the inactive examples, the term "called" is often preceded by an additional descriptor (e.g., "also called" or "sometimes called"), or the association between the preceding concept and the term "called" is not as direct or immediate. This difference in structure or phrasing may be the reason why the attention head remains inactive in these cases.'

In [10]:
import activating_dataset, prompt_generators
from importlib import reload
reload(activating_dataset)
reload(prompt_generators)
# Load Dataset
from activating_dataset import ActivatingDataset
from prompt_generators import GenerationPrompt

all_neuron_heads = list(nh_to_exp.keys())

eg_generation_prompts = {}

for neuron_head in all_neuron_heads:
    explanation_str = nh_to_exp[neuron_head].replace("\n", "")
    token = neuron_to_token[(neuron_head[0], neuron_head[1])]
    prompt_gen = GenerationPrompt(explanation_str, token)
    gpt_4_prompt = prompt_gen.get_prompt(num_examples=10, generate_negative=True)
    eg_generation_prompts[neuron_head] = gpt_4_prompt

In [12]:
print(eg_generation_prompts[(31, 3621, 468)])

We are studying attention heads in a transformer architecture neural network. Each attention head looks for some particular thing in a short document.
The attention head being studied helps to predict that the next token is " only", but it is only active in some documents and not others.
Explanation: """
In particular, this attention head is active when the document contains a statement or fact that is limited or restricted by the term "only". In the active examples, the attention head is highlighting a specific context or scenario where something is exclusive or unique. For example, "Excersire can only work if you are following a proper diet" implies that exercise alone is not enough.In the inactive examples, the term "only" is often used to emphasize the sole purpose or aspect of something rather than indicating exclusivity or a limitation in a broader context. For example, "for one reason and one reason only" uses "only" to stress that there is just one reason, but it doesn't indica

## Run GPT-4 to generate examples

In [14]:
import json
import os
filename = "data/eg_generation_1.jsonl"

if os.path.isfile(filename):
    raise Exception("File already exists!")

jobs = [
            {"model":"gpt-4",
            "messages":[{"role": "user", "content": gpt_4_prompt}],
            "max_tokens":800, 
        } for gpt_4_prompt in eg_generation_prompts.values()]

with open(filename, "w") as f:
    for job in jobs:
        json_string = json.dumps(job)
        f.write(json_string + "\n")

In [ ]:
import time
now = int(time.time())
print(now)

#########################################################################################
# WHEN YOU UPDATE THE CELL, REMEMBER TO UPDATE THE JSONL FILE NAME IF YOU'VE CHANGED IT #
#########################################################################################
current_time = 1686792857

if current_time + 20 < now: # Sanity check to make sure you don't spam this cell
    raise Exception("Update the current_time variable to be able to run this cell! Copy and paste the number above.")
else:
    print("all gucci")
    !python3 api_request_parallel_processor.py --requests_filepath data/eg_generation_1.jsonl --request_url https://api.openai.com/v1/chat/completions --max_requests_per_minute 100 --max_tokens_per_minute 20000
    # ^ This is very scary because the stdout looks like it sends repeated requests for the same thing so just run it in terminal


In [15]:
# Load the results and make the {neuron_head: response} dictionary

with open('data/eg_generation_1_results.jsonl', 'r') as json_file:
    raw_explanations = list(json_file)

prompt_to_response = {}
neuron_head_to_response = {}
neuron_head_to_prompt = eg_generation_prompts
neuron_heads = list(neuron_head_to_prompt.keys())

for i, json_str in enumerate(raw_explanations):
    result = json.loads(json_str)
    prompt = result[0]["messages"][0]["content"]
    response = result[1]["choices"][0]["message"]["content"]
    prompt_to_response[prompt] = response

for neuron_head in neuron_heads:
    prompt = neuron_head_to_prompt[neuron_head]
    response = prompt_to_response[prompt]
    neuron_head_to_response[neuron_head] = response

with open("data/eg_generation_1_nh_to_exp.json", "w") as f:
    nh_to_response_str_key = {str(k):v for k,v in neuron_head_to_response.items()}
    json.dump(nh_to_response_str_key, f)

## Create Prompts For Iteration

In [29]:
# import reload from wherever
from importlib import reload
import prompt_generators
reload(prompt_generators)
from prompt_generators import IterationPromptGen
from neuron_functions import get_head_attribution

In [17]:
with open("data/eg_generation_1_nh_to_exp.json") as f:
    generated_examples_str_key = json.load(f)
    generated_examples = {tuple(int(value.strip()) for value in k.strip('()').split(',')): v for k, v in generated_examples_str_key.items()}

In [32]:
check_if_responses_can_be_processed = False # Do this first
iteration_prompts = {}
iteration_prompts_ref = {}

for i, (neuron_head, generated_example) in tqdm(enumerate(generated_examples.items())):
    neuron, head_no = (neuron_head[0], neuron_head[1]), neuron_head[2]
    relevant_token = neuron_to_token[neuron]

    capitalised_token = relevant_token[0] + relevant_token[1].capitalize() + relevant_token[2:] # we can do this because by definition we got " word" tokens
    
    if 'is_inactive' in generated_example:
        generated_example = generated_example.replace('is_inactive', 'is inactive') # For some reason this was in one prompt

    # Extract the positive and negative examples. 
    def get_list_of_strings(string):
        return [s[3:] if s[1] == "." else s[4:] for s in string.split("\n") if s != ''] # Wonky but it works

    positive_examples = generated_example.split("Examples where the attention head is active:")[1].split("Examples where the attention head is inactive:")[0]
    negative_examples = generated_example.split("Examples where the attention head is inactive:")[1]
    positive_examples = get_list_of_strings(positive_examples)
    negative_examples = get_list_of_strings(negative_examples)

    # Cut off the examples at the last instance of <token>. They may happen more than once.
    positive_examples = [eg[:max(eg.rfind(relevant_token), eg.rfind(capitalised_token))] for eg in positive_examples]
    negative_examples = [eg[:max(eg.rfind(relevant_token), eg.rfind(capitalised_token))] for eg in negative_examples]

    if not check_if_responses_can_be_processed:
        # Load and Truncate Prompts (slightly cursed way of string -> token -> truncated tokens -> string)
        trunc_prompts = positive_examples + negative_examples
        # Run head attribution
        tokens = model.to_tokens(trunc_prompts, prepend_bos=True).to(device=device)
        original_logits, cache = model.run_with_cache(tokens, )

        # # Prepare prompts by heads
        head_attribution = get_head_attribution(model, cache, tokens, neuron)
        _, top_heads = torch.topk(head_attribution, k=3, dim=-1)
        top_heads_list = top_heads.tolist()

        top_heads_positive = top_heads_list[:len(positive_examples)]
        top_heads_negative = top_heads_list[len(positive_examples):]

        positive_examples_correct = []
        negative_examples_correct = []

        for i, example in enumerate(positive_examples):
            if head_no in top_heads_positive[i]:
                positive_examples_correct.append(example)
            else:
                negative_examples_correct.append(example)

        for i, example in enumerate(negative_examples):
            if head_no in top_heads_negative[i]:
                positive_examples_correct.append(example)
            else:
                negative_examples_correct.append(example)

        explanation_str = nh_to_exp[neuron_head].replace("\n", "")
        iter_gen = IterationPromptGen(positive_examples, negative_examples, positive_examples_correct, negative_examples_correct, relevant_token, explanation_str)
        iteration_prompt = iter_gen.get_prompt()

        iteration_prompts[neuron_head] = iteration_prompt
        iteration_prompts_ref[neuron_head] = [positive_examples, negative_examples, positive_examples_correct, negative_examples_correct]

0it [00:00, ?it/s]

Tried to stack head results when they weren't cached. Computing head results now


1it [00:04,  4.67s/it]

Tried to stack head results when they weren't cached. Computing head results now


2it [00:09,  4.66s/it]

...

174it [09:38,  2.74s/it]

Tried to stack head results when they weren't cached. Computing head results now


175it [09:40,  2.33s/it]

Tried to stack head results when they weren't cached. Computing head results now


176it [09:41,  3.31s/it]


In [36]:
print(iteration_prompts[(31, 3621, 468)])

The following solutions are the output of a Bayesian reasoner which is optimized to explain the function of attention heads in a neural network using limited evidence. Each attention head looks for some particular thing in a short passage.
The reasoner is trying to revise the explanation for an attention head that helps the model to predict that the next token is " only".
The current explanation is: This attention head is active when the document <explanation>contains a statement or fact that is limited or restricted by the term "only". In the active examples, the attention head is highlighting a specific context or scenario where something is exclusive or unique. For example, "Excersire can only work if you are following a proper diet" implies that exercise alone is not enough.In the inactive examples, the term "only" is often used to emphasize the sole purpose or aspect of something rather than indicating exclusivity or a limitation in a broader context. For example, "for one reason 

In [33]:
with open("data/iteration_prompts_1.json", "w") as f:
    iteration_prompts_str_key = {str(k):v for k,v in iteration_prompts.items()}
    json.dump(iteration_prompts_str_key, f)

In [34]:
import pickle
with open("data/iteration_prompts_ref_1.pkl", "wb") as f:
    pickle.dump(iteration_prompts_ref, f) 

# ref is original, ref_1 is with token at end of example

In [37]:
positive_lengths = []
accuracies = []
for head, (a, b, c, d) in iteration_prompts_ref.items():
    correct_positive = 0
    for eg in a:
        if eg in c:
            correct_positive += 1
    accuracies.append(correct_positive/len(a))
    positive_lengths.append(len(c))

print(sum(accuracies)/len(accuracies))
scatter(np.arange(len(accuracies)), accuracies)
scatter(np.arange(len(positive_lengths)), positive_lengths)


0.4059659090909092


In [38]:
print(list(iteration_prompts.values())[14])

The following solutions are the output of a Bayesian reasoner which is optimized to explain the function of attention heads in a neural network using limited evidence. Each attention head looks for some particular thing in a short passage.
The reasoner is trying to revise the explanation for an attention head that helps the model to predict that the next token is " together".
The current explanation is: This attention head is active when the document <explanation>mentions the concept of things or elements coming together or being brought together, but not in cases where the phrase "taken together" is used or where the sense of combining is not directly expressed. It seems to be sensitive to the specific way in which the idea of joining or merging is conveyed in the text, and not just the general notion of togetherness.</explanation>
With the explanation, the reasoner categorises the following examples of documents that activate this attention head:
Examples where the attention head is 

In [40]:
import json
import os
filename = "data/iteration_prompt_1.jsonl"

if os.path.isfile(filename):
    raise Exception("File already exists!")

jobs = [
            {"model":"gpt-4",
            "messages":[{"role": "user", "content": gpt_4_prompt}],
            "max_tokens":200, 
        } for gpt_4_prompt in iteration_prompts.values()]

with open(filename, "w") as f:
    for job in jobs:
        json_string = json.dumps(job)
        f.write(json_string + "\n")

In [ ]:
import time
now = int(time.time())
print(now)

#########################################################################################
# WHEN YOU UPDATE THE CELL, REMEMBER TO UPDATE THE JSONL FILE NAME IF YOU'VE CHANGED IT #
#########################################################################################
current_time = 1686792857

if current_time + 20 < now: # Sanity check to make sure you don't spam this cell
    raise Exception("Update the current_time variable to be able to run this cell! Copy and paste the number above.")
else:
    print("all gucci")
    !python3 api_request_parallel_processor.py --requests_filepath data/iteration_prompt_1.jsonl --request_url https://api.openai.com/v1/chat/completions --max_requests_per_minute 100 --max_tokens_per_minute 20000
    # ^ This is very scary because the stdout looks like it sends repeated requests for the same thing so just run it in terminal


In [42]:
# Load the results and make the {neuron_head: response} dictionary

with open('data/iteration_prompt_1_results.jsonl', 'r') as json_file:
    raw_explanations = list(json_file)

prompt_to_response = {}
neuron_head_to_response = {}
neuron_head_to_prompt = iteration_prompts
neuron_heads = list(neuron_head_to_prompt.keys())

for i, json_str in enumerate(raw_explanations):
    result = json.loads(json_str)
    prompt = result[0]["messages"][0]["content"]
    response = result[1]["choices"][0]["message"]["content"]
    prompt_to_response[prompt] = response

for neuron_head in neuron_heads:
    prompt = neuron_head_to_prompt[neuron_head]
    response = prompt_to_response[prompt]
    neuron_head_to_response[neuron_head] = response


with open("data/iteration_prompt_1_nh_to_exp.json", "w") as f:
    nh_to_response_str_key = {str(k):v for k,v in neuron_head_to_response.items()}
    json.dump(nh_to_response_str_key, f)

In [58]:
# Then run it a second time
all_neuron_heads = list(neuron_head_to_response.keys())

generation_prompts_v2 = {}

for neuron_head in all_neuron_heads:
    explanation_str = neuron_head_to_response[neuron_head].replace("\n", "")
    try:
        explanation_str = explanation_str.split("</explanation>")[0]
        print(explanation_str)
    except:
        continue
    token = neuron_to_token[(neuron_head[0], neuron_head[1])]
    prompt_gen = GenerationPrompt(explanation_str.replace("\n", ""), token)
    gpt_4_prompt = prompt_gen.get_prompt(num_examples=10, generate_negative=True)
    generation_prompts_v2[neuron_head] = gpt_4_prompt

contains a statement or fact where the term "only" is used to emphasize exclusivity, limitation, or a specific condition. In the active examples, the attention head is highlighting scenarios where something is restricted or unique due to the presence of the term "only". For example, "To unlock the secret level, you must collect all the hidden keys. The door will open only" implies that meeting a particular condition is required for the door to open. In the inactive examples, the term "only" is often used to emphasize the single aspect, purpose, or focus of something without necessarily indicating a broader context restriction or exclusivity. For example, "Sarah had one goal and one goal only" uses "only" to stress that there is just one goal but doesn't indicate a unique situation that the attention head looks for.
contains a statement or fact where the term "only" is used to emphasize exclusivity, limitation, or a specific condition. In the active examples, the attention head is highl

In [44]:
i = 0

In [50]:
i+=1
print(list(generation_prompts_v2.values())[i])

We are studying attention heads in a transformer architecture neural network. Each attention head looks for some particular thing in a short document.
The attention head being studied helps to predict that the next token is " number", but it is only active in some documents and not others.
Explanation: """
In particular, this attention head is active when the document contains a context or phrase that is related to record-breaking, chart-topping, or being highly ranked in terms of popularity or achievement, often in the music domain. The attention head is inactive when the document contains numbers related to rankings, positions, or sequences in other domains, as well as when it involves identification, phone numbers, parts, approvals, or other non-ranking contexts. The active examples usually involve quantifying the success or popularity of a subject, entity, or performance.</explanation>With the explanation, the reasoner categorizes the following examples of documents that activate t

In [62]:
import json
filename = "data/eg_generation_iter_1.jsonl"
jobs = [
            {"model":"gpt-4",
            "messages":[{"role": "user", "content": gpt_4_prompt}],
            "max_tokens":800,
        } for gpt_4_prompt in generation_prompts_v2.values()]

with open(filename, "w") as f:
    for job in jobs:
        json_string = json.dumps(job)
        f.write(json_string + "\n")

In [ ]:
import time
now = int(time.time())
print(now)

#########################################################################################
# WHEN YOU UPDATE THE CELL, REMEMBER TO UPDATE THE JSONL FILE NAME IF YOU'VE CHANGED IT #
#########################################################################################
current_time = 1686792857

if current_time + 20 < now: # Sanity check to make sure you don't spam this cell
    raise Exception("Update the current_time variable to be able to run this cell! Copy and paste the number above.")
else:
    print("all gucci")
    !python3 api_request_parallel_processor.py --requests_filepath data/eg_generation_iter_1.jsonl --request_url https://api.openai.com/v1/chat/completions --max_requests_per_minute 100 --max_tokens_per_minute 20000
    # ^ This is very scary because the stdout looks like it sends repeated requests for the same thing so just run it in terminal


In [64]:
# Load the results and make the {neuron_head: response} dictionary

with open('data/eg_generation_iter_1_results.jsonl', 'r') as json_file:
    raw_explanations = list(json_file)

prompt_to_response = {}
neuron_head_to_response = {}
neuron_head_to_prompt = generation_prompts_v2
neuron_heads = list(neuron_head_to_prompt.keys())

for i, json_str in enumerate(raw_explanations):
    result = json.loads(json_str)
    prompt = result[0]["messages"][0]["content"]
    response = result[1]["choices"][0]["message"]["content"]
    prompt_to_response[prompt] = response

for neuron_head in neuron_heads:
    prompt = neuron_head_to_prompt[neuron_head]
    response = prompt_to_response[prompt]
    neuron_head_to_response[neuron_head] = response


with open("data/eg_generation_iter_1_nh_to_exp.json", "w") as f:
    nh_to_response_str_key = {str(k):v for k,v in neuron_head_to_response.items()}
    json.dump(nh_to_response_str_key, f)

In [71]:
check_if_responses_can_be_processed = False # Do this first
iteration_prompts = {}
iteration_prompts_ref = {}

for i, (neuron_head, generated_example) in tqdm(enumerate(neuron_head_to_response.items())):
    neuron, head_no = (neuron_head[0], neuron_head[1]), neuron_head[2]
    relevant_token = neuron_to_token[neuron]

    capitalised_token = relevant_token[0] + relevant_token[1].capitalize() + relevant_token[2:] # we can do this because by definition we got " word" tokens
    
    if 'is_inactive' in generated_example:
        generated_example = generated_example.replace('is_inactive', 'is inactive') # For some reason this was in one prompt

    # Extract the positive and negative examples. 
    def get_list_of_strings(string):
        return [s[3:] if s[1] == "." else s[4:] for s in string.split("\n") if s != ''] # Wonky but it works

    positive_examples = generated_example.split("Examples where the attention head is active:")[1].split("Examples where the attention head is inactive:")[0]
    negative_examples = generated_example.split("Examples where the attention head is inactive:")[1]
    positive_examples = get_list_of_strings(positive_examples)
    negative_examples = get_list_of_strings(negative_examples)

    # Cut off the examples at the last instance of <token>. They may happen more than once.
    positive_examples = [eg[:max(eg.rfind(relevant_token), eg.rfind(capitalised_token))] for eg in positive_examples]
    negative_examples = [eg[:max(eg.rfind(relevant_token), eg.rfind(capitalised_token))] for eg in negative_examples]

    if not check_if_responses_can_be_processed:
        # Load and Truncate Prompts (slightly cursed way of string -> token -> truncated tokens -> string)
        trunc_prompts = positive_examples + negative_examples
        # Run head attribution
        tokens = model.to_tokens(trunc_prompts, prepend_bos=True).to(device=device)
        original_logits, cache = model.run_with_cache(tokens, )

        # # Prepare prompts by heads
        head_attribution = get_head_attribution(model, cache, tokens, neuron)
        _, top_heads = torch.topk(head_attribution, k=3, dim=-1)
        top_heads_list = top_heads.tolist()

        top_heads_positive = top_heads_list[:len(positive_examples)]
        top_heads_negative = top_heads_list[len(positive_examples):]

        positive_examples_correct = []
        negative_examples_correct = []

        for i, example in enumerate(positive_examples):
            if head_no in top_heads_positive[i]:
                positive_examples_correct.append(example)
            else:
                negative_examples_correct.append(example)

        for i, example in enumerate(negative_examples):
            if head_no in top_heads_negative[i]:
                positive_examples_correct.append(example)
            else:
                negative_examples_correct.append(example)

        explanation_str = nh_to_exp[neuron_head].replace("\n", "")
        iter_gen = IterationPromptGen(positive_examples, negative_examples, positive_examples_correct, negative_examples_correct, relevant_token, explanation_str)
        iteration_prompt = iter_gen.get_prompt()

        iteration_prompts[neuron_head] = iteration_prompt
        iteration_prompts_ref[neuron_head] = [positive_examples, negative_examples, positive_examples_correct, negative_examples_correct]

0it [00:00, ?it/s]

Tried to stack head results when they weren't cached. Computing head results now


1it [00:07,  7.45s/it]

Tried to stack head results when they weren't cached. Computing head results now


2it [00:10,  4.98s/it]

...

174it [10:30,  3.09s/it]

Tried to stack head results when they weren't cached. Computing head results now


175it [10:31,  2.69s/it]

Tried to stack head results when they weren't cached. Computing head results now


176it [10:35,  3.61s/it]


In [72]:
import pickle
with open("data/generated_prompts_v2_1.pkl", "wb") as f:
    pickle.dump(iteration_prompts_ref, f)

# ref is original, ref_1 is with token at end of example

In [1]:
import pickle
with open("data/generated_prompts_v2_1.pkl", "rb") as f:
    v2 = pickle.load(f)

with open("data/iteration_prompts_ref_1.pkl", "rb") as f:
    v1 = pickle.load(f)

In [33]:
accuracies = []
accuracies_positive = {}
accuracies_negative = []
f1s = []
precisions = []
recalls = []
for head, (pred_pos, pred_neg, correct_pos, correct_neg) in v2.items():
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    for eg in pred_pos:
        if eg in correct_pos:
            true_pos += 1
        else:
            false_pos += 1
    for eg in pred_neg:
        if eg in correct_neg:
            true_neg += 1
        else:
            false_neg += 1
    accuracy = (true_pos + true_neg) / (true_pos + true_neg + false_pos + false_neg)
    accuracy_positive = true_pos / (true_pos + false_pos)
    accuracy_negative = true_neg / (true_neg + false_neg)
    precision = true_pos / (true_pos + false_pos)
    try:
        recall = true_pos / (true_pos + false_neg)
        recalls.append(recall)
    except ZeroDivisionError:
        pass
    precisions.append(precision)

    # accuracies_positive.append(accuracy_positive)
    accuracies_positive[head] = accuracy_positive
    accuracies_negative.append(accuracy_negative)
    accuracies.append(accuracy)
    f1 = 2 * true_pos / (2 * true_pos + false_pos + false_neg)
    f1s.append(f1)



print(np.mean(accuracies))
print(np.mean(list(accuracies_positive.values())))
print(np.mean(accuracies_negative))
print(np.mean(f1s))
print(np.mean(precisions))
print(np.mean(recalls))

0.5954229797979799
0.4034090909090909
0.7856534090909091
0.40573377829807455
0.4034090909090909
0.6660497535647056


In [31]:
scatter(np.arange(len(accuracies_positive)), list(accuracies_positive.values()), hover_name=labels) # add hover